In [6]:
!pip install --extra-index-url pip.repos.neuron.amazonaws.com https://pip.repos.neuron.amazonaws.com/torch-xla/torch_xla-1.11.0%2Btorchneuron3-cp38-cp38-linux_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 MB 9.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch-xla
    Found existing installation: torch-xla 1.0
    Uninstalling torch-xla-1.0:
      Successfully uninstalled torch-xla-1.0


In [17]:
!pip install torch==1.11.0

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 1.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 101.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 84.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0
    Uninstalling torch-2.1.0:
      Successfully uninstalled torch-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.16.0 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.


In [1]:
import csv
from datasets import Dataset, DatasetDict
import logging
import os
import pandas as pd
from time import gmtime, strftime, perf_counter
from tqdm.auto import tqdm
import torch
import torch_xla.core.xla_model as xm
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler

/root/micromamba/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: /root/micromamba/lib/python3.8/site-packages/_XLAC.cpython-38-x86_64-linux-gnu.so: undefined symbol: _ZNK3c1010TensorImpl7stridesEv

In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "True"

In [3]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
model_name = "bert-base-cased"
## define xla as device for using AWS Trainium Neuron Cores
device = "xla"
batch_size = 8
num_epochs = 6

In [5]:
def tokenize_and_encode(data):
    results = tokenizer(data["text"], padding="max_length", truncation=True)
    return results

In [6]:
!ls

bert-trainium.ipynb  data  models  no_trainium.py  trainium.py


In [7]:
path = os.path.abspath("data")
model_path = os.path.abspath("models")
csv_path = path + "/train.csv"

In [8]:
train = pd.read_csv(
    csv_path,
    sep=',',
    quotechar='"',
    quoting=csv.QUOTE_ALL,
    escapechar='\\',
    encoding='utf-8'
)
train.head(4)

,text,labels
0,are we headed toward dystopia? #thesocialdilemma,1
1,"have you seen #thesocialdilemma, on it is quit...",1
2,what platform are you addicted to?#thesocialdi...,0
3,cant recommend this documentary enough to anyo...,1


In [9]:
train["labels"].value_counts()

2    7370
1    5321
0    2772
Name: labels, dtype: int64

In [10]:
train_dataset = Dataset.from_dict(train)
hg_dataset = DatasetDict({"train": train_dataset})

In [11]:
## Loading Hugging Face AutoTokenizer for the defined model
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=model_path)
ds_encoded = hg_dataset.map(tokenize_and_encode, batched=True, remove_columns=["text"])

Map: 100%|█████████████████████████████████████████████████████████████████████| 15463/15463 [00:02<00:00, 6118.23 examples/s]


In [12]:
ds_encoded.set_format("torch")

In [13]:
## Creating a DataLoader object for iterating over it during the training epochs
train_dl = DataLoader(ds_encoded["train"], shuffle=True, batch_size=batch_size)

In [14]:
## Loading Hugging Face pre-trained model for sequence classification for the defined model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
#os.environ["PJRT_DEVICE"] = "TPU"

In [16]:
model.to(device)

RuntimeError: torch_xla/csrc/runtime/runtime.cc:23 : $PJRT_DEVICE is not set.


In [ ]:
optimizer = AdamW(model.parameters(), lr=1.45e-4)

num_training_steps = num_epochs * len(train_dl)
progress_bar = tqdm(range(num_training_steps))
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

logger.info("Start training: {}".format(strftime("%Y-%m-%d %H:%M:%S", gmtime())))

In [ ]:
## Start model training and defining the training loop
model.train()
for epoch in range(num_epochs):
    for batch in train_dl:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        ## xm.mark_step is executing the current graph, updating the model params, and notifiy end of step to Neuron Core
        xm.mark_step()
        optimizer.zero_grad()
        progress_bar.update(1)

    logger.info("Epoch {}, rank {}, Loss {:0.4f}".format(epoch, xm.get_ordinal(), loss.detach().to("cpu")))

logger.info("End training: {}".format(strftime("%Y-%m-%d %H:%M:%S", gmtime())))

## Using XLA for saving model after training for being sure only one copy of the model is saved
os.makedirs("./../../models/checkpoints/{}".format(current_timestamp), exist_ok=True)
checkpoint = {"state_dict": model.state_dict()}
xm.save(checkpoint, "./../../models/checkpoints/{}/checkpoint.pt".format(current_timestamp))